# Complex Geometry

In [ ]:
import sinaps as sn
import numpy as np

## Creating Neuron structure from swc file

We choose a neuron from the [neuromorpho](http://neuromorpho.org) database :

In [ ]:
filename = "http://neuromorpho.org/dableFiles/chang/CNG%20version/V247fs-MT-Untreated-56.CNG.swc"

Use the function read_swc from io module to create a neuron from this file :

In [ ]:
nrn = sn.io.read_swc(filename)
nrn.plot()

## Accessing data

The neuron structure is stored using the [networkx](https://networkx.org) library, and you can access this underlying structure using :

In [ ]:
nrn.graph

For example :

In [ ]:
nrn.graph.edges

In [ ]:
nrn.graph.nodes

You can access the section corresponding to an edge in the graph structure using the `section` attribute of edges

In [ ]:
nrn.graph[1][2]['section']

### Setting custom radius

We want to redifine the radius of the dentrite, assumiong that the radius decreases at each bifurcation

In [ ]:
def set_radius(G,dep,a,factor):
    if G.degree(dep)>1:
        a=max(a*(G.degree(dep)-1)**(factor),0.1)
        for node in G[dep]:
            s = G[dep][node]['section']
            if s.a == 1000:
                s.a=a
                set_radius(G,node,a,factor)

In [ ]:
nrn['dendrite'].a = 1000

In [ ]:
# Setting radius of each node
set_radius(nrn.graph,1,5,-0.5) 

In [ ]:
nrn.plot()

In [ ]:
nrn.plot.layout(force=True)
nrn.plot()

In [ ]:
s = nrn['soma']
[nrn.sections[s] for s in s]

In [ ]:
import random

In [ ]:
# Finding the leaves
leaves = nrn.leaves()
del leaves[2]
del leaves[3] # Removing the soma
stim_leaves = random.sample(leaves,10)
stim_sec = nrn[stim_leaves]

### Setting up the channels

In [ ]:
nrn[:].add_channel(sn.channels.Hodgkin_Huxley())
stim_sec.add_channel(sn.channels.NMDAR(0.5,gnmda=2),1)

In [ ]:
# Stimulate leaves
stim_sec

### Running the simu

In [ ]:
# Initialisation of the simulation
sim=sn.Simulation(N,dx=100)

In [ ]:
# Runing the simulation
sim.run((0,1000))

In [ ]:
# Data structure (panda series)
sim.V

### Plots

In [ ]:
# Plotting the potential
sim.plot()

In [ ]:
# Extracting some sections
sim['Section0060'].plot() * sim['Section0000'].plot()

### Chemical reactions

In [ ]:
# Clearing previous reactions
N.reactions=[]
# Adding Ca + BF <-> BFB (BF = GCamp, BFB = GCamp-Ca)
N.add_reaction(
    {Ca:1,
     BF:1},
    {BFB:1},
    k1=12.3,
    k2=0.002)
# Calcium extrusion
N.add_reaction(
    {Ca:1},
    {},
    k1=0.003,k2=0)
#Calcium initial concentration
for s in N:
    s.C0[Ca]=10**(-7)
#Buffer initial concentration
for s in N:
    s.C0[BF]=10**(-6)

In [ ]:
# Running the chemical reactions part   
sim.run_diff(max_step=1)

### Plots

In [ ]:
# Plot of the Calcium concentration
sim[:].plot.C(Ca)

In [ ]:
# Plot of GCamp bound with Ca
sim[:].plot.C(BFB)

In [ ]:
# 2D plots, non linear in time (plotting against dt in the simulations) Highlits time periods with activity. Careful: the graph don't corresponds in the x axis, as the simulation for the potential and for the chemical reactions are separated.
( graph2D(sim.V[dfs]).opts(title='Potential',    **common_opts)
+ graph2D(sim.C[Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2D(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
# 2D graph linear in time
(graph2Dlinear(sim.V[dfs]).opts(title='Potential', **common_opts)
+ graph2Dlinear(sim.C[Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2Dlinear(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
# 2D graph linear in time, extracting the first 6 ms
(graph2Dlinear(sim.V[0:6][dfs]).opts(title='Potential', **common_opts)
+ graph2Dlinear(sim.C[0:6][Ca][dfs]).opts(title='Calcium',      **common_opts)
+ graph2Dlinear(sim.C[0:6][BFB][dfs]).opts(title='Bound Calcium',**common_opts)
    )

In [ ]:
graph2Dlinear(sim.C[BFB][dfs]).opts(title='Bound Calcium',**common_opts)

In [ ]:
Test=sim.C[BFB][:].iloc
N[:].name
for name in N[:].name:
    sim.C[BFB][:].iloc[:,0]/N[name].a
    sim.C[BFB][:].iloc[:,1]/N[name].a

In [ ]:
Test=sim.C[BFB].values


In [ ]:
sim.C[BFB][:].iloc[:,1]

## Currents

In [ ]:
# Currents in the simulation
[ch.__name__ for ch in sim.channels]

In [ ]:
I(sim,'Hodgkin_Huxley')['Section0484'].iloc[0:550,0].hvplot()

In [ ]:
I(sim,'Hodgkin_Huxley_Ca')['Section0484'].iloc[0:600,0].hvplot()

In [ ]:
I(sim,'NMDAR')['Section0236'].clip(0).iloc[0:600,1].hvplot()

In [ ]:
graph2Dlinear(I(sim,'Hodgkin_Huxley')[dfs][0:4]).opts(**common_opts,title='HH Current')

In [ ]:
graph2Dlinear(I(sim,'NMDAR')[dfs][0:4].clip(0)).opts(**common_opts,title='NMDA Current')